# Data Cleaning and Preprocessing

**Objective:** Clean both datasets, handle missing values, standardize model names, and prepare data for analysis.

In [1]:
import pandas as pd
import numpy as np 
import re
import os

pd.set_option('display.max_columns', None)

print("Libraries imported successfully.")

Libraries imported successfully.


## Load Raw Data

In [3]:
# Load both datasets
df_arena = pd.read_csv('../data/raw/chatbot_arena.csv')
df_kaggle = pd.read_csv('../data/raw/llm_comparison_dataset.csv')

print(f"Arena: {df_arena.shape}, Kaggle: {df_kaggle.shape}")

Arena: (57477, 9), Kaggle: (200, 15)


## Inspect Columns Names

See what columns we're working with to identify model names, preferences, and prompts.

In [4]:
print("Arena columns:")
print(df_arena.columns.tolist())

Arena columns:
['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie']


In [5]:
print("Kaggle columns:")
print(df_kaggle.columns.tolist())

Kaggle columns:
['Model', 'Provider', 'Context Window', 'Speed (tokens/sec)', 'Latency (sec)', 'Benchmark (MMLU)', 'Benchmark (Chatbot Arena)', 'Open-Source', 'Price / Million Tokens', 'Training Dataset Size', 'Compute Power', 'Energy Efficiency', 'Quality Rating', 'Speed Rating', 'Price Rating']


## Understand the Winner Columns

The Arena dataset has three winner columns (`winner_model_a`, `winner_model_b`, `winner_tie`).

In [ ]:
# Check unique values in winner columns
print("winner_model_A unique values:", df_arena['winner_model_a'].unique())
print("winner_model_B unique values:", df_arena['winner_model_b'].unique())
print("winner_tie unique values:", df_arena['winner_tie'].unique())

winner_model_A unique values: [1 0]
winner_model_B unique values: [0 1]
winner_tie unique values: [0 1]


In [7]:
# Check how many rows have each winner type
print(" Rows where model_a won:", df_arena['winner_model_a'].sum())
print(" Rows where model_b won:", df_arena['winner_model_b'].sum())
print(" Rows where tie occurred:", df_arena['winner_tie'].sum())

 Rows where model_a won: 20064
 Rows where model_b won: 19652
 Rows where tie occurred: 17761


## Create Single Winner Column

Convert the three winner columns into one categorical column for easier analysis.

In [8]:
# Create a single 'winner' column
def determine_winner(row):
    if row['winner_model_a'] == 1:
        return 'model_a'
    elif row['winner_model_b'] == 1:
        return 'model_b'
    elif row['winner_tie'] == 1:
        return 'tie'
    else:
        return 'unknown'
    
df_arena['winner'] = df_arena.apply(determine_winner, axis=1)

# Check distribution
print(df_arena['winner'].value_counts())

winner
model_a    20064
model_b    19652
tie        17761
Name: count, dtype: int64


## Check Model Names
Let's see what models we have in each dataset.

In [10]:
# Unique models in Arena dataset
arena_models = set(df_arena['model_a'].unique()) | set(df_arena['model_b'].unique())
print(f"Unique models in Arena: {len(arena_models)}")
print("\n Sample Arena models (first 10):")
for i, model in enumerate(list(arena_models)[:10], 1):
    print(f"{i}. {model}")

Unique models in Arena: 64

 Sample Arena models (first 10):
1. gpt-4-0314
2. claude-2.0
3. openchat-3.5-0106
4. wizardlm-13b
5. tulu-2-dpo-70b
6. dolphin-2.2.1-mistral-7b
7. zephyr-7b-alpha
8. RWKV-4-Raven-14B
9. yi-34b-chat
10. llama2-70b-steerlm-chat


In [11]:
# Unique models in Kaggle dataset
kaggle_models = set(df_kaggle['Model'].unique())
print(f"Unique models in Kaggle: {len(kaggle_models)}")
print("\nSample Kaggle models (first 10):")
for i, model in enumerate(list(kaggle_models)[:10], 1):
    print(f"  {i}. {model}")

Unique models in Kaggle: 70

Sample Kaggle models (first 10):
  1. Mistral-3
  2. Command-3
  3. Claude-2
  4. Claude-9
  5. Llama-3
  6. Gemini-4
  7. Gemini-3
  8. Nova-7
  9. Nova-8
  10. Llama-9


## Find Common Models

Check how many models overlap between datasets.

In [ ]:
# Find common models
common_models = arena_models & kaggle_models
arena_only = arena_models - kaggle_models
kaggle_only = kaggle_models - arena_models

print(f"Models in both datasets: {len(common_models)}")
print(f"Models only in Arena: {len(arena_only)}")
print(f"Models only in Kaggle: {len(kaggle_only)}")
print(f"Merge success rate: {len(common_models) / len(arena_models) * 100:.1f}%")

Models in both datasets: 0
Models only in Arena: 64
Models only in Kaggle: 70
Merge success rate: 0.0%


In [13]:
# Show some examples of common models
if common_models:
    print("Sample common models:")
    for i, model in enumerate(list(common_models)[:10], 1):
        print(f"  {i}. {model}")

In [14]:
# Show examples of models only in Arena (might need name standardization)
if arena_only:
    print("\nSample Arena-only models (first 10):")
    for i, model in enumerate(list(arena_only)[:10], 1):
        print(f"  {i}. {model}")


Sample Arena-only models (first 10):
  1. gpt-4-0314
  2. solar-10.7b-instruct-v1.0
  3. claude-2.0
  4. mpt-30b-chat
  5. vicuna-33b
  6. nous-hermes-2-mixtral-8x7b-dpo
  7. openchat-3.5-0106
  8. falcon-180b-chat
  9. wizardlm-13b
  10. llama-13b


In [15]:
# Show examples of models only in Kaggle (might need name standardization)
if kaggle_only:
    print("\nSample Kaggle-only models (first 10):")
    for i, model in enumerate(list(kaggle_only)[:10], 1):
        print(f"  {i}. {model}")


Sample Kaggle-only models (first 10):
  1. Command-3
  2. Mistral-3
  3. Claude-2
  4. Claude-9
  5. Llama-3
  6. Gemini-4
  7. Gemini-3
  8. Nova-7
  9. Nova-8
  10. Llama-9


## Full Model Lists
Let's see all models to create a standardization mapping.

In [ ]:
# Show all Arena models
print("All Arena models:")
print("=" * 70)
arena_models_sorted = sorted(list(arena_models))
for i, model in enumerate(arena_models_sorted, 1):
    print(f"{i: 3d}. {model}")
print(f"\nTotal: {len(arena_models_sorted)} models")

All Arena models:
  1. RWKV-4-Raven-14B
  2. alpaca-13b
  3. chatglm-6b
  4. chatglm2-6b
  5. chatglm3-6b
  6. claude-1
  7. claude-2.0
  8. claude-2.1
  9. claude-instant-1
 10. codellama-34b-instruct
 11. deepseek-llm-67b-chat
 12. dolly-v2-12b
 13. dolphin-2.2.1-mistral-7b
 14. falcon-180b-chat
 15. fastchat-t5-3b
 16. gemini-pro
 17. gemini-pro-dev-api
 18. gpt-3.5-turbo-0125
 19. gpt-3.5-turbo-0314
 20. gpt-3.5-turbo-0613
 21. gpt-3.5-turbo-1106
 22. gpt-4-0125-preview
 23. gpt-4-0314
 24. gpt-4-0613
 25. gpt-4-1106-preview
 26. gpt4all-13b-snoozy
 27. guanaco-33b
 28. koala-13b
 29. llama-13b
 30. llama-2-13b-chat
 31. llama-2-70b-chat
 32. llama-2-7b-chat
 33. llama2-70b-steerlm-chat
 34. mistral-7b-instruct
 35. mistral-7b-instruct-v0.2
 36. mistral-medium
 37. mixtral-8x7b-instruct-v0.1
 38. mpt-30b-chat
 39. mpt-7b-chat
 40. nous-hermes-2-mixtral-8x7b-dpo
 41. oasst-pythia-12b
 42. openchat-3.5
 43. openchat-3.5-0106
 44. openhermes-2.5-mistral-7b
 45. palm-2
 46. pplx-70b-on

In [18]:
# Show all Kaggle models
print("ALL KAGGLE MODELS:")
print("="*70)
kaggle_models_sorted = sorted(list(kaggle_models))
for i, model in enumerate(kaggle_models_sorted, 1):
    print(f"{i:3d}. {model}")
print(f"\nTotal: {len(kaggle_models_sorted)} models")

ALL KAGGLE MODELS:
  1. Claude-1
  2. Claude-2
  3. Claude-3
  4. Claude-5
  5. Claude-6
  6. Claude-7
  7. Claude-8
  8. Claude-9
  9. Command-1
 10. Command-2
 11. Command-3
 12. Command-4
 13. Command-5
 14. Command-6
 15. Command-7
 16. Command-8
 17. Command-9
 18. DeepSeek-1
 19. DeepSeek-2
 20. DeepSeek-3
 21. DeepSeek-4
 22. DeepSeek-5
 23. DeepSeek-6
 24. DeepSeek-7
 25. DeepSeek-8
 26. DeepSeek-9
 27. GPT-1
 28. GPT-2
 29. GPT-3
 30. GPT-4
 31. GPT-5
 32. GPT-6
 33. GPT-7
 34. GPT-8
 35. GPT-9
 36. Gemini-1
 37. Gemini-2
 38. Gemini-3
 39. Gemini-4
 40. Gemini-5
 41. Gemini-6
 42. Gemini-7
 43. Gemini-8
 44. Gemini-9
 45. Llama-1
 46. Llama-2
 47. Llama-3
 48. Llama-4
 49. Llama-5
 50. Llama-6
 51. Llama-7
 52. Llama-8
 53. Llama-9
 54. Mistral-1
 55. Mistral-2
 56. Mistral-3
 57. Mistral-4
 58. Mistral-5
 59. Mistral-6
 60. Mistral-7
 61. Mistral-8
 62. Nova-1
 63. Nova-2
 64. Nova-3
 65. Nova-4
 66. Nova-5
 67. Nova-6
 68. Nova-7
 69. Nova-8
 70. Nova-9

Total: 70 models


## Analysis Update

**Issue:** The Kaggle dataset contains sythetic model names that don't match the real Arena models. <br>
**New Approach:** Focus the analysis on the Arena dataset (which has real user data) and skip the merge with Kaggle. There is still 57,000+ converstions with 64 real models.

## Handle Missing Values - Arena Dataset

In [20]:
# Check missing values
missing = df_arena.isnull().sum()
missing_pct = (missing / len(df_arena) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})

print("Missing values in Arena dataset:")
print(missing_df[missing_df['Missing Count'] > 0])

Missing values in Arena dataset:
Empty DataFrame
Columns: [Missing Count, Missing %]
Index: []


## Remove Rows with Missing Critical Data
Drop rows where prompt or model names are missing.

In [21]:
# Check how many rows have missing critical data
critical_cols = ['model_a', 'model_b', 'prompt']
rows_before = len(df_arena)

for col in critical_cols:
    missing_count = df_arena[col].isnull().sum()
    print(f"{col}: {missing_count} missing ({missing_count/rows_before*100:.2f}%)")

model_a: 0 missing (0.00%)
model_b: 0 missing (0.00%)
prompt: 0 missing (0.00%)


In [22]:
# Drop rows with missing critical columns
df_arena_clean = df_arena.dropna(subset=critical_cols)

rows_after = len(df_arena_clean)
rows_dropped = rows_before - rows_after

print(f"\nRows before: {rows_before:,}")
print(f"Rows after: {rows_after:,}")
print(f"Rows dropped: {rows_dropped:,} ({rows_dropped/rows_before*100:.2f}%)")


Rows before: 57,477
Rows after: 57,477
Rows dropped: 0 (0.00%)


## Remove Duplicate Conversations

In [23]:
# Check for duplicates based on prompt and models
duplicates = df_arena_clean.duplicated(subset=['prompt', 'model_a', 'model_b'])
num_duplicates = duplicates.sum()

print(f"Duplicate conversations: {num_duplicates:,} ({num_duplicates/len(df_arena_clean)*100:.2f}%)")

Duplicate conversations: 126 (0.22%)


In [24]:
# Remove duplicates
df_arena_clean = df_arena_clean.drop_duplicates(subset=['prompt', 'model_a', 'model_b'])

print(f"Rows after removing duplicates: {len(df_arena_clean):,}")

Rows after removing duplicates: 57,351


## Create Prompt Length Features

In [25]:
# Calculate prompt length (characters)
df_arena_clean['prompt_length'] = df_arena_clean['prompt'].str.len()

# Calculate word count
df_arena_clean['prompt_word_count'] = df_arena_clean['prompt'].str.split().str.len()

print("Prompt length statistics:")
print(df_arena_clean['prompt_length'].describe())
print("\nWord count statistics:")
print(df_arena_clean['prompt_word_count'].describe())

Prompt length statistics:
count    57351.000000
mean       368.842130
std       1073.962642
min          7.000000
25%         52.000000
50%         96.000000
75%        242.000000
max      33056.000000
Name: prompt_length, dtype: float64

Word count statistics:
count    57351.000000
mean        53.940594
std        143.983150
min          1.000000
25%          9.000000
50%         16.000000
75%         39.000000
max       4719.000000
Name: prompt_word_count, dtype: float64


## Create Response Length Features

In [26]:
# Response lengths
df_arena_clean['response_a_length'] = df_arena_clean['response_a'].str.len()
df_arena_clean['response_b_length'] = df_arena_clean['response_b'].str.len()

print("Response A length:")
print(df_arena_clean['response_a_length'].describe())
print("\nResponse B length:")
print(df_arena_clean['response_b_length'].describe())

Response A length:
count    57351.000000
mean      1379.571446
std       1514.849236
min          4.000000
25%        409.000000
50%       1079.000000
75%       1864.000000
max      54058.000000
Name: response_a_length, dtype: float64

Response B length:
count    57351.000000
mean      1387.991386
std       1538.790085
min          4.000000
25%        414.000000
50%       1088.000000
75%       1874.000000
max      53830.000000
Name: response_b_length, dtype: float64


## Save Cleaned Dataset

In [27]:
# Save cleaned dataset
output_path = '../data/processed/arena_cleaned.csv'
df_arena_clean.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to: {output_path}")
print(f"Shape: {df_arena_clean.shape}")
print(f"Columns: {df_arena_clean.columns.tolist()}")

Cleaned dataset saved to: ../data/processed/arena_cleaned.csv
Shape: (57351, 14)
Columns: ['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'winner', 'prompt_length', 'prompt_word_count', 'response_a_length', 'response_b_length']


In [30]:
# Summary of cleaning
print("Cleaning Summary:")
print("="*70)
print(f"Original rows: {rows_before:,}")
print(f"After removing missing values: {rows_after:,}")
print(f"After removing duplicates: {len(df_arena_clean):,}")
print(f"Final dataset: {len(df_arena_clean):,} rows * {df_arena_clean.shape[1]} columns")
print(f"Data retention: {len(df_arena_clean)/rows_before*100:.1f}%")

Cleaning Summary:
Original rows: 57,477
After removing missing values: 57,477
After removing duplicates: 57,351
Final dataset: 57,351 rows * 14 columns
Data retention: 99.8%
